# Deep Agents

<img src="./media/deep_agent_diagram.png" width=400>

Language model based applications have moved from simple chat interfaces, to more 'agentic' frameworks. While the definition of what an 'agent' is varies, it generally refers to a system where an LLM is equipped with some tools, and is allowed to operate continuously. It is often with the intent to complete a more complex goal by chaining together multiple actions together, ultimately relying on the reasoning abilities of the language model to determine what steps to take and when to stop.

This initial approach has worked well, with agentic systems proving more useful, helpful, and capable than their naive chat completions beginnings, ushering in a new wave of interest and investment into arming LLMs with various integrations and tools to be able to perform more complex tasks. Although useful, there still remained a disconnect between the capability of an LLM with digitial tools and a human with the same tools, primarily around 'long-horizon' tasks.

<img src="./media/models-are-succeeding-at-increasingly-long-tasks.png" width=600>

[Measuring AI Ability to Complete Long Tasks](https://metr.org/blog/2025-03-19-measuring-ai-ability-to-complete-long-tasks/)

When observing a graph that compares success rates of models on human benchmarked time to complete tasks, most modern day models can complete tasks that often take 4 minutes for humans to complete successfully, however as task complexity grows success rate falls. Anecdotely, not many _useful_ tasks tend to fall into the 'takes four minutes to complete' category, i.e:

<img src="./media/easy_tasks.png" width=450>

[HCAST: Human-Calibrated Autonomy Software Tasks](https://metr.org/hcast.pdf)

With most of what would be considered 'productive' tasks often taking much more time and effort to complete. 

As LLMs become smarter and more capable through improved training techniques, their ability for successful long-horizon task completion has improved (reference prior graph), but even more gains have been made through custom frameworks meant to encourage and enable this kind of behavior. Popularized initially by workflows such as [Deep Research](https://blog.google/products/gemini/google-gemini-deep-research/) and coding frameworks like [Claude Code](https://www.anthropic.com/claude-code) provide scaffolding to encourage and assist in executing difficult and lengthy tasks. 

<img src="./media/da_la.png" width=400>

[Deep Agents](https://blog.langchain.com/deep-agents/)

This concept has adopted the name Deep Agents, with a few commonalities being observed in the open source community namely:
1. A detailed and specific **system prompt**
2. A **planning** or to-do list tool
3. A **file system** for context management
4. **Sub Agents!**

In this notebook, we'll discuss the theory behind why these techniques have proven successful for creating Deep Agents and ecouraging longer task execution, and provide an example of this all coming together by using open source frameworks to create our own Deep Agent!

## System Prompt

<img src="./media/system_prompt.png" width=300>

More robust and long running agents tend to have very detailed and beautifully written system prompts, some recent snapshots of popular agent prompts can be seen here:

- [Claude Code System Prompt](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Claude%20Code/claude-code-system-prompt.txt)
- [Cursor Agent System Prompt](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Cursor%20Prompts/Agent%20Prompt%202025-09-03.txt)
- [Manus System Prompt](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Manus%20Agent%20Tools%20%26%20Prompt/Prompt.txt)

Notably, these prompts are roughly 200+ lines of code long (not counting tool instructions!), with specific instructions around tone, style, behavior, and tool use- often all blending together.

Claude Code provides pseudo examples, with placeholders indicating different actions and future behavior.

```
<example>
user: Help me write a new feature that allows users to track their usage metrics and export them to various formats

assistant: I'll help you implement a usage metrics tracking and export feature. Let me first use the TodoWrite tool to plan this task.
Adding the following todos to the todo list:
1. Research existing metrics tracking in the codebase
2. Design the metrics collection system
3. Implement core metrics tracking functionality
4. Create export functionality for different formats

Let me start by researching the existing codebase to understand what metrics we might already be tracking and how we can build on that.

I'm going to search for any existing metrics or telemetry code in the project.

I've found some existing telemetry code. Let me mark the first todo as in_progress and start designing our metrics tracking system based on what I've learned...

[Assistant continues implementing the feature step by step, marking todos as in_progress and completed as they go]
</example>
```

Manus includes styles and values

```
## Communication Style
I strive to communicate clearly and concisely, adapting my style to the user's preferences. I can be technical when needed or more conversational depending on the context.

## Values I Uphold
- Accuracy and reliability in information
- Respect for user privacy and data
- Ethical use of technology
- Transparency about my capabilities
- Continuous improvement
```

Cursor Agent has a whole coding style guide:

```
<code_style>
IMPORTANT: The code you write will be reviewed by humans; optimize for clarity and readability. Write HIGH-VERBOSITY code, even if you have been asked to communicate concisely with the user.

Naming
Avoid short variable/symbol names. Never use 1-2 character names
Functions should be verbs/verb-phrases, variables should be nouns/noun-phrases
Use meaningful variable names as described in Martin's "Clean Code":
Descriptive enough that comments are generally not needed
Prefer full words over abbreviations
Use variables to capture the meaning of complex conditions or operations
Examples (Bad → Good)
genYmdStr → generateDateString
n → numSuccessfulRequests
[key, value] of map → [userId, user] of userIdToUser
resMs → fetchUserDataResponseMs
Static Typed Languages
Explicitly annotate function signatures and exported/public APIs
Don't annotate trivially inferred variables
Avoid unsafe typecasts or types like any
Control Flow
Use guard clauses/early returns
Handle error and edge cases first
Avoid unnecessary try/catch blocks
NEVER catch errors without meaningful handling
Avoid deep nesting beyond 2-3 levels
Comments
Do not add comments for trivial or obvious code. Where needed, keep them concise
Add comments for complex or hard-to-understand code; explain "why" not "how"
Never use inline comments. Comment above code lines or use language-specific docstrings for functions
Avoid TODO comments. Implement instead
Formatting
Match existing code style and formatting
Prefer multi-line over one-liners/complex ternaries
Wrap long lines
Don't reformat unrelated code </code_style>
```

We can learn a lot from these prompting best practices of successful tools:
1. **Prompts can be long and complex as long as they are clear and organized**: Each of these system prompts are much longer than simple applications tend to process. Despite this almost no space is wasted and there is a clear hierarchy of introducing the agent and overall functionality at the beginning, setting tone and behavior, then explicitly describing all available functionality with clear examples delineated by XML tags.
2. **Few-Shot Prompting**: Providing examples is used heavily to reinforce behavior and proper tool execution. These are usually not super exact examples (i.e. copying full conversations) but provide higher level overviews with placeholders for actions (eerily similar to `*Does an Action*` style internet writing).
3. **Annotated Tools**: Tool use and descriptions are kept mostly to their schemas, i.e. [Claude Code Tools](https://github.com/x1xhlol/system-prompts-and-models-of-ai-tools/blob/main/Claude%20Code/claude-code-tools.json) and show both good and bad examples (back to few shot prompting!)
4. **Not All Technical**: Prompting is still (semi) vibes based, important points are reiterated multiple times at multiple different lines, 1st person (i.e. I am XYZ) and 3rd person (You are XYZ) perspectives are used in different tools, some prompts are more uniform (like cursor) than others (manus)

Overall, your prompts should be detailed, complete, and outline the scenario and cases your agent will find itself in with clear directions and guidance towards operating. Length is not a concern as long as quality is there!

## Planning Tool

<img src="./media/todo_list.png" width=300>

One consistent emphasis with almost all long running deep agents is the inclusion of a planning tool or todo list tool that the model is encouraged to create and maintain. Using this tool is often one of the first actions an agent is encouraged to do. Once the todo list has been made, the individual entries are flagged as incomplete, in progress, or completed. 

While reasoning through plans has been proven to increase task performance for the last few years (i.e. [Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models](https://arxiv.org/pdf/2305.04091)) the focus and reason for this is actually more of tactical than just encouraging planning behavior. It's biggest benefit is pushing the current objective and goal to the forefront of the model's context. This helps keep the model on track and understand where it sits within the current action sequence. 

<img src="./media/manus_context.png" width=600>

[Context Engineering for AI Agents: Lessons from Building Manus](https://manus.im/blog/Context-Engineering-for-AI-Agents-Lessons-from-Building-Manus)

A keen eye may notice that the execution of the todo list tool doesn't really _do_ anything in the sense of a traditionally defined tool. This is intentional, as using the tool and resurfacing/reiterating the plan keeps the model on track as historical context grows.

## File System

<img src="./media/file_system.png" width=300>

As agents run longer, take more actions, and ingest more information the amount of context that needs to be managed quickly balloons. While modern language model's boast impressive [1 Million+ token context windows](https://ai.google.dev/gemini-api/docs/models#gemini-2.5-pro), it is not best practice to actively exploit this. Operating environments may contain massive amounts of information that can't feasibly fit into a context window (i.e. multimillion loc repos), and increasingly reingesting all context adds unnecessary costs/latency to the system. 

To address this limitation and allow context to be kept but not loaded at all times, files and links are used as a form of 'memory'. 

<img src="./media/manus_file_system.png" width=600>

[Context Engineering for AI Agents: Lessons from Building Manus](https://manus.im/blog/Context-Engineering-for-AI-Agents-Lessons-from-Building-Manus)

An agent is provided with ways of creating, writing, editing, and searching (i.e. grep/semantic) over files. Using these loads the relevant file information into context, which can then be offloaded when not needed but maintain a file path, URL, or URI to reference in conversation history. This uniquely provides a memory system that is compressable, restorable, and persistent across the run or runs of an agent. As a plus, the same capabilities let agents like Cursor or Claude Code navigate and understand massive codebases without needing to load every single piece of information. 

## Sub Agents

<img src="./media/sub_agents.png" width=300>

The final, and potentially most powerful, technique used with deep agents are **sub agents**, or what can be thought of as the subroutine of agentic systems. Sub agents are nothing more than specialized individual tool calling agents that can be invoked by the main deep agent system. In the case of Claude Code, sub agents are kicked off via a `Task` function call, where the specific agent is chosen and passed generated instructions that are relevant and contextual to completing the overarching goal at hand.

The use of sub agents provide, context management, modularity, specialization and efficiency benefits:
- **Context Management**: Each sub agent operates within its own context window, allowing it to focus on just the task at hand with the tools it has. This lets it pass only the most important context back to the deep agent system in it's final answer without requiring the entire context history be tracked in the main state. This approach is purely an 'agent as a tool' style.
- **Modularity**: A single sub agent can be defined and then used in multiple scenarios, i.e. a web research sub agent can be applied to any deep agent system that would require some form of web research.
- **Specialization**: Defining task specific sub agents improves performance at those specific tasks over a generalized system. Sub agents can use more narrow prompting and custom built for the job tools to more directly tackle a request. 
- **Efficicency**: On top of the aforementioned context/token management that this provides, sub agents can be ran in parallel to concurrently progress on multiple outstanding tasks. 

Sub agents compliment the approach of task decomposition encouraged via the todo tool and streamline the overall operation of a deep agent system by giving it a form of delegation.

<img src="./media/claude_subagents.png" width=600>

[Build with Claude Code - Subagents](https://docs.anthropic.com/en/docs/claude-code/sub-agents)

_Side Note_: Anthropic has been developing on this idea since their introduction of sub agents in [Claude 3 Opus as an economic analyst](https://www.youtube.com/watch?v=sjL6Gl6ZIqs) over a year ago!